<a href="https://colab.research.google.com/github/sinaparajita/Brain-tumor-detection-/blob/main/SURE4JOB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)


[[ 6.76569194e-02  6.34959713e-02  4.87131476e-02  7.93049783e-02
   3.74480933e-02  2.65277131e-03  3.93749513e-02 -7.09846430e-03
   5.93614057e-02  3.15369777e-02  6.00981042e-02 -5.29052615e-02
   4.06067781e-02 -2.59308387e-02  2.98428647e-02  1.12688600e-03
   7.35148042e-02 -5.03817983e-02 -1.22386612e-01  2.37028562e-02
   2.97265332e-02  4.24768738e-02  2.56337579e-02  1.99516211e-03
  -5.69190793e-02 -2.71598026e-02 -3.29035036e-02  6.60248846e-02
   1.19007230e-01 -4.58791181e-02 -7.26214051e-02 -3.25840116e-02
   5.23413308e-02  4.50553037e-02  8.25298298e-03  3.67024504e-02
  -1.39415674e-02  6.53918684e-02 -2.64271945e-02  2.06370896e-04
  -1.36643145e-02 -3.62810642e-02 -1.95043720e-02 -2.89738011e-02
   3.94270122e-02 -8.84091184e-02  2.62427772e-03  1.36713730e-02
   4.83062901e-02 -3.11566498e-02 -1.17329173e-01 -5.11690713e-02
  -8.85288343e-02 -2.18963325e-02  1.42986095e-02  4.44167666e-02
  -1.34815648e-02  7.43392259e-02  2.66382862e-02 -1.98763069e-02
   1.79191

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6528e-03,  3.9375e-02, -7.0985e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1269e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9727e-02,  4.2477e-02,  2.5634e-02,  1.9952e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2530e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0637e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6243e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3482e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4718e

In [8]:
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# ✅ Define sample job description and candidate info
job_description = "We are looking for a backend developer with experience in Python, Django, and REST APIs."
profile = "Backend Developer with Python and Flask experience"
designation = "Software Engineer"

# ✅ Combine candidate info
candidate_text = profile + " " + designation

# ✅ Encode
job_embedding = model.encode(job_description, convert_to_tensor=True)
candidate_embedding = model.encode(candidate_text, convert_to_tensor=True)

# ✅ Calculate similarity
similarity = util.cos_sim(job_embedding, candidate_embedding).item() * 100

print(f"Similarity Score: {similarity:.2f}%")


Similarity Score: 77.86%


In [9]:
from sentence_transformers import SentenceTransformer, util

# Load lightweight model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample usage
job_embedding = model.encode(job_description, convert_to_tensor=True)
candidate_text = profile + " " + designation
candidate_embedding = model.encode(candidate_text, convert_to_tensor=True)

# Compute similarity
similarity = util.cos_sim(job_embedding, candidate_embedding).item() * 100


In [13]:
candidate = {
    "profile": "Backend Developer with Python and Flask experience",
    "currentDesignation": "Software Engineer",
    "totalExperience": "3",
    "currentCTC": "700000"
}


In [15]:
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample data
job_description = "We are hiring a backend developer with strong Python and Django experience."

candidate = {
    "profile": "Experienced backend engineer skilled in Python and Flask.",
    "currentDesignation": "Software Engineer",
    "totalExperience": "4",
    "currentCTC": "600000"
}

# Combine candidate text
candidate_text = candidate["profile"] + " " + candidate["currentDesignation"]

# Encode both
job_embedding = model.encode(job_description, convert_to_tensor=True)
candidate_embedding = model.encode(candidate_text, convert_to_tensor=True)

# Cosine similarity
similarity = util.cos_sim(job_embedding, candidate_embedding).item() * 100
print(f"Similarity Score: {similarity:.2f}%")


Similarity Score: 73.21%


In [31]:
import json
from sentence_transformers import SentenceTransformer, util

# Load the lightweight model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Load candidate data from JSON
with open("candidate_sample.json", "r") as f:
    data = json.load(f)

job_description = data["job_description"]
resumes = data["resumes"]

# Step 2: Encode the job description
job_embedding = model.encode(job_description, convert_to_tensor=True)

# Step 3: Process each candidate
shortlisted = []

for candidate in resumes:
    full_name = " ".join(filter(None, [
        candidate.get("firstName", ""),
        candidate.get("middleName", ""),
        candidate.get("lastName", "")
    ])).strip()

    profile = candidate.get("profile", "")
    designation = candidate.get("currentDesignation", "")
    candidate_text = profile + " " + designation

    candidate_embedding = model.encode(candidate_text, convert_to_tensor=True)
    similarity = util.cos_sim(job_embedding, candidate_embedding).item() * 100

    if similarity >= 60:
        shortlisted.append({
            "candidateId": candidate.get("candidateId", -1),
            "name": full_name,
            "similarity_score": round(similarity, 2),
            "profile": profile,
            "designation": designation
        })

# Step 4: Sort candidates by similarity score (descending)
shortlisted_sorted = sorted(shortlisted, key=lambda x: x["similarity_score"], reverse=True)

# Step 5: Print final sorted candidates
print("\n=== Ranked Candidates (Similarity ≥ 60%) ===\n")
for idx, c in enumerate(shortlisted_sorted, 1):
    print(f"{idx}. {c['name']} – Score: {c['similarity_score']}% – {c['designation']}")



=== Ranked Candidates (Similarity ≥ 60%) ===



In [32]:
import json
from sentence_transformers import SentenceTransformer, util

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Load candidate data from JSON
with open("candidate_sample.json", "r") as f:
    data = json.load(f)

job_description = data["job_description"]
resumes = data["resumes"]

# Step 2: Encode job description
job_embedding = model.encode(job_description, convert_to_tensor=True)

# Step 3: Process all candidates (no threshold)
ranked_candidates = []

for candidate in resumes:
    full_name = " ".join(filter(None, [
        candidate.get("firstName", ""),
        candidate.get("middleName", ""),
        candidate.get("lastName", "")
    ])).strip()

    profile = candidate.get("profile", "")
    designation = candidate.get("currentDesignation", "")
    candidate_text = profile + " " + designation

    candidate_embedding = model.encode(candidate_text, convert_to_tensor=True)
    similarity = util.cos_sim(job_embedding, candidate_embedding).item() * 100

    ranked_candidates.append({
        "candidateId": candidate.get("candidateId", -1),
        "name": full_name,
        "similarity_score": round(similarity, 2),
        "profile": profile,
        "designation": designation
    })

# Step 4: Sort all candidates by similarity score (descending)
ranked_candidates.sort(key=lambda x: x["similarity_score"], reverse=True)

# Step 5: Display the results
print("\n=== Ranked Candidates by Similarity ===\n")
for i, c in enumerate(ranked_candidates, 1):
    print(f"{i}. {c['name']} - Score: {c['similarity_score']}%")
    print(f"   Designation: {c['designation']}")
    print(f"   Profile: {c['profile']}\n")



=== Ranked Candidates by Similarity ===

1. Priya Verma - Score: 49.0%
   Designation: HR Specialist
   Profile: HR Recruiter

2. Sneha Gupta - Score: 48.75%
   Designation: Senior HR IT Recruiter
   Profile: HR Recruiter

3. Sushma Gupta - Score: 48.63%
   Designation: Hr Recruiter
   Profile: Hr Recruiter

4. Simmi Kumari - Score: 45.87%
   Designation: HR Associate
   Profile: HR Recruiter

5. Megha Tyagi - Score: 45.87%
   Designation: HR Associate
   Profile: HR Recruiter

6. Rupal Yadav - Score: 45.84%
   Designation: HR Executive
   Profile: HR Recruiter

7. Tanya Agarwal - Score: 45.84%
   Designation: HR Executive
   Profile: HR Recruiter

8. Naina Sharma - Score: 43.59%
   Designation: Associate Recruiter 
   Profile: HR Recruiter

9. Sakshita Srivastava - Score: 41.84%
   Designation: soft skill trainer 
   Profile: HR Recruiter

